# Applying many similar aggregations.

Now suppose we want to compute a large number of similar aggregations, one for each of a collection of columns.  A similar approach will allow us to accomplish this task regardless of the number of columns.

In [1]:
import polars as pl

## Data set

We will be using two of the data sets provided by the Museam of Modern Art (MoMA) in this lecture.  Make sure that you have downloaded each repository.  [Download Instructions](./get_MOMA_data.ipynb)

#### MoMA Artwork

**Note.** The last two columns should also be of type `pl.Float`.  Let's define a schema using a `dict` comprehension.

In [2]:
artwork = (pl.read_csv("./data/Artworks.csv", infer_schema_length=50000)
          )
artwork.head(2)

Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,Dimensions,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,f64,f64,f64,f64,f64,f64,f64,str,str
"""Ferdinandsbrücke Project, Vien…","""Otto Wagner""","""6210""","""(Austrian, 1841–1918)""","""(Austrian)""","""(1841)""","""(1918)""","""(Male)""","""1896""","""Ink and cut-and-pasted painted…","""19 1/8 x 66 1/2"" (48.6 x 168.9…","""Fractional and promised gift o…","""885.1996""","""Architecture""","""Architecture & Design""","""1996-04-09""","""Y""",2,"""http://www.moma.org/collection…","""http://www.moma.org/media/W1si…",null,null,null,48.6,null,null,168.9,null,null
"""City of Music, National Superi…","""Christian de Portzamparc""","""7470""","""(French, born 1944)""","""(French)""","""(1944)""","""(0)""","""(Male)""","""1987""","""Paint and colored pencil on pr…","""16 x 11 3/4"" (40.6 x 29.8 cm)""","""Gift of the architect in honor…","""1.1995""","""Architecture""","""Architecture & Design""","""1995-01-17""","""Y""",3,"""http://www.moma.org/collection…","""http://www.moma.org/media/W1si…",null,null,null,40.6401,null,null,29.8451,null,null


###  Comprehensions EVERYWHERE!

**Comprehension mindset.** Whenever creating a `list`/`tuple`/`dict`/`set`, automate the process with a comprehension.

**Current task.** Create a schema `dict` that correctly sets all dimension columns to `float`.

**Strategy.** Use a comprehension and the fact all dimension columns end in `')'` to create the schema.

In [3]:
artwork_schema = {col: pl.Float64 if col.endswith(')') else pl.String for col in artwork.columns}

artwork_schema

{'Title': String,
 'Artist': String,
 'ConstituentID': String,
 'ArtistBio': String,
 'Nationality': String,
 'BeginDate': String,
 'EndDate': String,
 'Gender': String,
 'Date': String,
 'Medium': String,
 'Dimensions': String,
 'CreditLine': String,
 'AccessionNumber': String,
 'Classification': String,
 'Department': String,
 'DateAcquired': String,
 'Cataloged': String,
 'ObjectID': String,
 'URL': String,
 'ThumbnailURL': String,
 'Circumference (cm)': Float64,
 'Depth (cm)': Float64,
 'Diameter (cm)': Float64,
 'Height (cm)': Float64,
 'Length (cm)': Float64,
 'Weight (kg)': Float64,
 'Width (cm)': Float64,
 'Seat Height (cm)': Float64,
 'Duration (sec.)': Float64}

In [4]:
artwork = (pl.read_csv("./data/Artworks.csv", schema=artwork_schema)
          )
artwork.head(2)

Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,Dimensions,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Ferdinandsbrücke Project, Vien…","""Otto Wagner""","""6210""","""(Austrian, 1841–1918)""","""(Austrian)""","""(1841)""","""(1918)""","""(Male)""","""1896""","""Ink and cut-and-pasted painted…","""19 1/8 x 66 1/2"" (48.6 x 168.9…","""Fractional and promised gift o…","""885.1996""","""Architecture""","""Architecture & Design""","""1996-04-09""","""Y""","""2""","""http://www.moma.org/collection…","""http://www.moma.org/media/W1si…",null,null,null,48.6,null,null,168.9,null,null
"""City of Music, National Superi…","""Christian de Portzamparc""","""7470""","""(French, born 1944)""","""(French)""","""(1944)""","""(0)""","""(Male)""","""1987""","""Paint and colored pencil on pr…","""16 x 11 3/4"" (40.6 x 29.8 cm)""","""Gift of the architect in honor…","""1.1995""","""Architecture""","""Architecture & Design""","""1995-01-17""","""Y""","""3""","""http://www.moma.org/collection…","""http://www.moma.org/media/W1si…",null,null,null,40.6401,null,null,29.8451,null,null


## A WET transformation - using alias

This solution is WET, not DRY, as all 9 of the expressions in mutate are very similar ... if only here was a way to perform these similar expressions all at once ...

In [5]:
# Input expressions as positional arguments.
(artwork
 .group_by('Classification')
 .agg(pl.col('Circumference (cm)').mean().alias('Mean Circumference (cm)'),
      pl.col('Depth (cm)').mean().alias('Mean Depth (cm)'),
      pl.col('Diameter (cm)').mean().alias('Mean Diameter (cm)'),
      pl.col('Height (cm)').mean().alias('Mean Height (cm)'),
      pl.col('Length (cm)').mean().alias('Mean Length (cm)'),
      pl.col('Weight (kg)').mean().alias('Mean Weight (kg)'),
      pl.col('Width (cm)').mean().alias('Mean Width (cm)'),
      pl.col('Seat Height (cm)').mean().alias('Mean Seat Height (cm)'),
      pl.col('Duration (sec.)').mean().alias('Mean Duration (sec.)'),
     )
)

Classification,Mean Circumference (cm),Mean Depth (cm),Mean Diameter (cm),Mean Height (cm),Mean Length (cm),Mean Weight (kg),Mean Width (cm),Mean Seat Height (cm),Mean Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Publication""",null,null,null,31.25,null,null,22.65,null,null
"""Digital""",null,null,null,null,null,null,null,null,4590.0
"""Mies van der Rohe Archive""",null,22.5,null,41.44568,null,90.7194,56.195844,null,null
"""Architecture""",null,39.439213,47.879818,46.990801,145.520935,11.04081,59.985583,null,2160.461538
"""Performance""",null,29.111299,null,130.39205,null,null,131.951426,null,600.0
…,…,…,…,…,…,…,…,…,…
"""Work on Paper""",null,1.209352,null,83.666016,null,null,78.392955,null,null
"""Print""",38.0,1.716388,49.066169,40.158467,14.5,1.5,37.848143,null,0.0
"""Textile""",null,8.724003,null,108.386216,103.0,null,118.780417,null,null


In [6]:
# Input a list of expressions
(artwork
 .group_by('Classification')
 .agg([pl.col('Circumference (cm)').mean().alias('Mean Circumference (cm)'),
       pl.col('Depth (cm)').mean().alias('Mean Depth (cm)'),
       pl.col('Diameter (cm)').mean().alias('Mean Diameter (cm)'),
       pl.col('Height (cm)').mean().alias('Mean Height (cm)'),
       pl.col('Length (cm)').mean().alias('Mean Length (cm)'),
       pl.col('Weight (kg)').mean().alias('Mean Weight (kg)'),
       pl.col('Width (cm)').mean().alias('Mean Width (cm)'),
       pl.col('Seat Height (cm)').mean().alias('Mean Seat Height (cm)'),
       pl.col('Duration (sec.)').mean().alias('Mean Duration (sec.)'),
      ],
     )
)

Classification,Mean Circumference (cm),Mean Depth (cm),Mean Diameter (cm),Mean Height (cm),Mean Length (cm),Mean Weight (kg),Mean Width (cm),Mean Seat Height (cm),Mean Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Audio""",NaN,0.17894,19.685039,11.84564,NaN,NaN,11.942762,NaN,1327.666667
"""Architectural Model""",NaN,8.890018,NaN,44.450089,NaN,NaN,44.450089,NaN,NaN
"""Frank Lloyd Wright Archive""",NaN,3.738871,26.670053,55.780718,NaN,NaN,70.306716,NaN,NaN
"""Performance""",NaN,29.111299,NaN,130.39205,NaN,NaN,131.951426,NaN,600.0
"""Drawing""",NaN,1.353477,72.246,43.344329,213.4,15.4223,45.738161,NaN,NaN
…,…,…,…,…,…,…,…,…,…
"""Publication""",NaN,NaN,NaN,31.25,NaN,NaN,22.65,NaN,NaN
"""Ephemera""",NaN,1.322905,4.75,27.317681,25.08255,NaN,24.492504,NaN,NaN
"""Photograph""",NaN,3.372236,8.041579,28.519292,NaN,226.798512,31.585365,NaN,280.0


In [7]:
help(artwork.group_by().agg)

Help on method agg in module polars.dataframe.group_by:

agg(*aggs: 'IntoExpr | Iterable[IntoExpr]', **named_aggs: 'IntoExpr') -> 'DataFrame' method of polars.dataframe.group_by.GroupBy instance
    Compute aggregations for each group of a group by operation.

    Parameters
    ----------
    *aggs
        Aggregations to compute for each group of the group by operation,
        specified as positional arguments.
        Accepts expression input. Strings are parsed as column names.
    **named_aggs
        Additional aggregations, specified as keyword arguments.
        The resulting columns will be renamed to the keyword used.

    Examples
    --------
    Compute the aggregation of the columns for each group.

    >>> df = pl.DataFrame(
    ...     {
    ...         "a": ["a", "b", "a", "b", "c"],
    ...         "b": [1, 2, 1, 3, 3],
    ...         "c": [5, 4, 3, 2, 1],
    ...     }
    ... )
    >>> df.group_by("a").agg(pl.col("b"), pl.col("c"))  # doctest: +IGNORE_RESULT
    s

In [8]:
compute_mean = lambda col: pl.col(col).mean().alias(f'Mean {col}')

In [9]:
# Input a list of expressions
(artwork
 .group_by('Classification')
 .agg([compute_mean(c) for c in artwork.columns if c.endswith(')')]
     )
)

Classification,Mean Circumference (cm),Mean Depth (cm),Mean Diameter (cm),Mean Height (cm),Mean Length (cm),Mean Weight (kg),Mean Width (cm),Mean Seat Height (cm),Mean Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Photography Research/Reference""",null,null,null,45.55,null,null,34.8,null,6120.0
"""Sculpture""",null,57.139591,106.085079,105.494561,465.3702,1387.632147,103.907527,null,1860.0
"""Architecture""",null,39.439213,47.879818,46.990801,145.520935,11.04081,59.985583,null,2160.461538
"""Media""",null,1.696429,null,12.47931,null,null,13.668966,null,null
"""Photograph""",null,3.372236,8.041579,28.519292,null,226.798512,31.585365,null,280.0
…,…,…,…,…,…,…,…,…,…
"""Publication""",null,null,null,31.25,null,null,22.65,null,null
"""Collage""",null,2.1875,null,70.355261,null,null,103.505515,null,null
"""Graphic Design""",null,0.0,null,326.597574,null,null,54.28382,null,null


In [10]:
# Unpacking positional arguments
(artwork
 .group_by('Classification')
 .agg(*[compute_mean(c) for c in artwork.columns if c.endswith(')')]
     )
)

Classification,Mean Circumference (cm),Mean Depth (cm),Mean Diameter (cm),Mean Height (cm),Mean Length (cm),Mean Weight (kg),Mean Width (cm),Mean Seat Height (cm),Mean Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Architectural Model""",NaN,8.890018,NaN,44.450089,NaN,NaN,44.450089,NaN,NaN
"""Film""",NaN,5.89144,NaN,6.097442,0.0,NaN,10.666683,NaN,31653.865471
"""Graphic Design""",NaN,0.0,NaN,326.597574,NaN,NaN,54.28382,NaN,NaN
"""Photography Research/Reference""",NaN,NaN,NaN,45.55,NaN,NaN,34.8,NaN,6120.0
"""Ephemera""",NaN,1.322905,4.75,27.317681,25.08255,NaN,24.492504,NaN,NaN
…,…,…,…,…,…,…,…,…,…
"""Work on Paper""",NaN,1.209352,NaN,83.666016,NaN,NaN,78.392955,NaN,NaN
"""Drawing""",NaN,1.353477,72.246,43.344329,213.4,15.4223,45.738161,NaN,NaN
"""Film (object)""",NaN,NaN,NaN,8.7,NaN,NaN,9.4,NaN,NaN


In [11]:
dimension_col_means = lambda df: [compute_mean(c) for c in df.columns if c.endswith(')')]

In [12]:
# List of input expressions
(artwork
 .group_by('Classification')
 .agg(dimension_col_means(artwork)
     )
)

Classification,Mean Circumference (cm),Mean Depth (cm),Mean Diameter (cm),Mean Height (cm),Mean Length (cm),Mean Weight (kg),Mean Width (cm),Mean Seat Height (cm),Mean Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Collage""",null,2.1875,null,70.355261,null,null,103.505515,null,null
"""Graphic Design""",null,0.0,null,326.597574,null,null,54.28382,null,null
"""Software""",null,null,null,null,null,null,null,null,null
"""Architecture""",null,39.439213,47.879818,46.990801,145.520935,11.04081,59.985583,null,2160.461538
"""Painting""",null,8.087142,138.357934,122.422894,null,96.285524,130.379904,null,null
…,…,…,…,…,…,…,…,…,…
"""Multiple""",29.516667,7.796307,9.303058,19.382265,26.65,2.783105,20.988888,null,60.0
"""Photography Research/Reference""",null,null,null,45.55,null,null,34.8,null,6120.0
"""Installation""",null,70.281632,136.220202,111.509097,582.520444,317.517917,116.929314,null,4917.898089


In [13]:
# Unpacking positional arguments
(artwork
 .group_by('Classification')
 .agg(*dimension_col_means(artwork)
     )
)

Classification,Mean Circumference (cm),Mean Depth (cm),Mean Diameter (cm),Mean Height (cm),Mean Length (cm),Mean Weight (kg),Mean Width (cm),Mean Seat Height (cm),Mean Duration (sec.)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Sculpture""",NaN,57.139591,106.085079,105.494561,465.3702,1387.632147,103.907527,NaN,1860.0
"""Film (object)""",NaN,NaN,NaN,8.7,NaN,NaN,9.4,NaN,NaN
"""Film""",NaN,5.89144,NaN,6.097442,0.0,NaN,10.666683,NaN,31653.865471
"""Digital""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4590.0
"""Mies van der Rohe Archive""",NaN,22.5,NaN,41.44568,NaN,90.7194,56.195844,NaN,NaN
…,…,…,…,…,…,…,…,…,…
"""Audio""",NaN,0.17894,19.685039,11.84564,NaN,NaN,11.942762,NaN,1327.666667
"""Architectural Model""",NaN,8.890018,NaN,44.450089,NaN,NaN,44.450089,NaN,NaN
"""Print""",38.0,1.716388,49.066169,40.158467,14.5,1.5,37.848143,NaN,0.0


## <font color="red"> Exercise 3.6</font>

Now suppose that we want to compute both the mean and standard deviation of each dimension.  Perform this task using an approach to the one shown above, this the results as DRY as possible. **Hint.** You can add two lists of transformations.

In [10]:
# Your code here